In [ ]:
class hazardMap:
    import pandas as pd
    import import_ipynb
    from generateimages import generateImages as gi
    import numpy as np
    import scipy
    from scipy import ndimage
    import scipy.misc
    from scipy.ndimage.measurements import label
    import skimage.measure   
    from osgeo import gdal
    import itertools

    
    def __init__(self, zip_code):
        #Addresses for reading address information, building footprint coordinates, and parcel outline coordinates
        root_path           = 'C:/Users/ntste/Documents/Insight/Tree Data/'
        root_image_path     = root_path + 'SAS Output/'
        generate_image_path = 'C:/Users/ntste/Documents/Insight/Tree Data/SAS Output/Code/generateImages.ipynb'
        address_info_path   = root_path + 'LA_County_Parcels_small.csv'
        footprint_info_path = root_path + 'Building_Footprints-shp/footprint_nodes.csv'
        parcel_info_path    = root_path + 'parcel_locations.csv'
        
        address_info   = pd.read_csv(address_info_path)
        footprint_info = pd.read_csv(footprint_info_path)
        parcel_info    = pd.read_csv(parcel_info_path,dtype={'PARCEL_WKT': str, 'AIN': str})
        
        #Format parcel info
        parcel_info = parcel_info.dropna()
        parcel_info = parcel_info[(parcel_info.AIN != 'nan')] #remove nan strings that aren't actual NaNs
        parcel_info = parcel_info[(parcel_info.AIN != ' ')]
        parcel_info = parcel_info[parcel_info.AIN.str.isnumeric()]
        parcel_info['AIN'] = parcel_info['AIN'].astype('int64')
        
        #perform inner join of all tables by AIN
        master_table = address_info.merge(footprint_info, how = 'inner', on = ['AIN'])
        master_table = master_table.merge(parcel_info, how = 'inner', on = ['AIN'])
        self.master_table = master_table.loc[master_table['SitusFullAddress'].str.contains(zip_code, na=False)]

    def image_by_address(self, address, imsize = 1024, classified = False):
        """
        Returns imsize x imsize RGB image centering around lot center location based on address
        Calls generateImages class
        master_table: joined table containing addresses, parcels, and building outlines
        address: string with full address
        imsize: side length of image in pixels
        """
        row        = self.master_table.loc[master_table['SitusFullAddress'] == address]
        center_lat = float(list(row['LAT_LON'])[0].split(',')[0])
        center_lon = float(list(row['LAT_LON'])[0].split(',')[1])

        imfo       = gi(root_image_path, center_lat, center_lon)
        image      = imfo.generate_image(classified = classified)

        return image, imfo, center_lat, center_lon, row
    
    def map_by_address(self,image, thresh, class_label = 5, imsize = 1024):
        """
        Returns imsize x imsize binary image centering around lot center location based on address
        Calls generateImages class; shows estimated tree locations
        image: input image with kmeans classifications
        class_label: class label in kmeans classified image (defaults to trees)
        imsize: side length of image in pixels
        """
        label_image = remove_small_clusters(image,thresh,class_label,class_pick = True)

        return label_image
    
     def remove_small_clusters(self, im, thresh, tree_class=5, class_pick = False):
        """
        Isolates tree-heavy class for kmeans classified image and 
        performs connected component analysis to remove components where
        N_pixels < thresh
        returns binary image of large tree clusters
        """
        if class_pick:
            isolated_class_image = im == tree_class
        else:
            isolated_class_image = im

        isolated_class_image.astype(int)
        structure            = np.ones((3,3), dtype = np.int)
        labeled,ncomponents  = label(isolated_class_image,structure)
        unique, counts       = np.unique(labeled, return_counts=True)
        chunks               = dict(zip(unique, counts))
        chunks               = {key:val for key, val in chunks.items() if ((val > thresh) and (val != max(chunks.values())))}
        output_map           = np.zeros_like(isolated_class_image)
        keys                 = [key for key in chunks]
        for i in keys:
            output_map[labeled == i] = 1

        return output_map
    
    def structures_by_address(self, address):
        """
        Returns list of containing the GPS locations of vertices outlining the main structure at a given address
        """
        row = self.master_table.loc[self.master_table['SitusFullAddress'] == address]
        wkt = list(row['WKT'])
        return wkt[0].split('MULTIPOLYGON (((')[1].split(')')[0].split(',')    
    
    def parcel_boundaries_by_address(self, address):
        """
        Returns list of containing the GPS locations of vertices outlining the property boundaries at a given address
        """
        row = self.master_table.loc[self.master_table['SitusFullAddress'] == address]
        wkt = list(row['PARCEL_WKT'])
        return wkt[0].split('MULTIPOLYGON (((')[1].split(')')[0].split(',')

    def convert_vertices_to_pixels(self, vertices, image_info, imsize = 1024):
        """
        Returns a list of tuples containing (x,y) location in pixels defining a polygon
        vertices: list of strings of lat, lon returned by structures_by_address
        """
        center_coordinates = image_info.center_coords
        ul_subset_x        = center_coordinates[0] - imsize/2. #x pixel location of upper left corner in subset image
        ul_subset_y        = center_coordinates[1] - imsize/2. #y pixel location of upper left corner in subset image

        vertex_coordinates = []
        driver = gdal.GetDriverByName('GTiff')
        filename = image_info.parent_image_path #path to raster
        dataset = gdal.Open(filename)
        band = dataset.GetRasterBand(1)
        cols = dataset.RasterXSize
        rows = dataset.RasterYSize
        transform = dataset.GetGeoTransform()
        xOrigin = transform[0]
        yOrigin = transform[3]
        pixelWidth = transform[1]
        pixelHeight = -transform[5]
        data = band.ReadAsArray(0, 0, cols, rows)
        points_list = [(float(vertex.split()[0]),float(vertex.split()[1])) for vertex in vertices] #list of X,Y coordinates
        
        for point in points_list:
            col = int((point[0] - xOrigin) / pixelWidth)
            row = int((yOrigin - point[1] ) / pixelHeight)
            vertex_coordinates.append((col-ul_subset_x, row-ul_subset_y))

        return vertex_coordinates   
    
    def structure_intersection_map(self, tree_map, structure_vertices, imsize = 1024):
        """
        Return boolean image showing regions where structures and trees overlap
        """
        import PIL.ImageDraw as ImageDraw
        import PIL.Image as Image
        import numpy as np

        poly_image = Image.new("RGB", (imsize,imsize))
        draw = ImageDraw.Draw(poly_image)
        draw.polygon(tuple(structure_vertices), fill = 2)
        overlap = np.array(poly_image)[:,:,0]*tree_map == 2

        return overlap, True in overlap

    def bresenham_line(self, x0, y0, x1, y1):
        """
        Returns a list of (x,y) tuples of pixel locations connecting the input points
        """
        steep = abs(y1 - y0) > abs(x1 - x0)
        if steep:
            x0, y0 = y0, x0  
            x1, y1 = y1, x1

        switched = False
        if x0 > x1:
            switched = True
            x0, x1 = x1, x0
            y0, y1 = y1, y0

        if y0 < y1: 
            ystep = 1
        else:
            ystep = -1

        deltax = x1 - x0
        deltay = abs(y1 - y0)
        error = -deltax / 2
        y = y0

        line = []    
        for x in range(x0, x1 + 1):
            if steep:
                line.append((y,x))
            else:
                line.append((x,y))

            error = error + deltay
            if error > 0:
                y = y + ystep
                error = error - deltax
        if switched:
            line.reverse()
        return line

    def get_parcel_pixels(self, parcel_coordinates):
        """
        Takes in parcel coordinates returned by convert_vertices_to_pixels
        Returns list of all (x,y) points connecting those vertices in a straight line
        """
        parcel_pixels = [bresenham_line(int(parcel_coordinates[i][0]), int(parcel_coordinates[i][1]), int(parcel_coordinates[i+1][0]), int(parcel_coordinates[i+1][1])) for i in range(len(parcel_coordinates)-1)]
        return list(itertools.chain.from_iterable(parcel_pixels))

    def get_parcel_image(self, parcel_pixels, imsize = 1024):
        """
        Input: parcel pixels from get_parcel_pixels, image size in pixels
        Returns imsize^2 nummpy array with parcel pixels as 2 and others as 0
        """
        parcel_image = np.zeros((imsize,imsize,3))
        for i in range(len(parcel_pixels)):
            parcel_image[parcel_pixels[i][1],parcel_pixels[i][0],:] = 2
            
        return parcel_image

    def parcel_intersection_map(self, tree_map, parcel_image, warn_padding, imfo):
        """
        Return boolean image showing regions where property lines and trees overlap
        tree_map: input map of tree locations
        parcel_image: input image with parcel lines
        warn_padding: distance to expand intersection flag (pixels)
        imfo: image info
        """
        parcel_image = scipy.ndimage.filters.convolve(parcel_image,np.ones((warn_padding,warn_padding,3)))
        parcel_image[parcel_image>0] = 2
        overlap = parcel_image[:,:,0]*tree_map == 2
        
        return overlap, True in overlap

    def address_suggestions(self, user_input, n_suggestions = 5, thresh = 0.2):
        """
        takes in user input string and returns top n_suggestions suggestions
        master_table: master dataframe that includes full addresses as SitusFullAddress
        user_input: user input string
        n_suggestions: maximum number of ranked suggestions returned
        thresh: similarity threshold below which no suggestions are returned
        """
        import difflib
        suggestions = difflib.get_close_matches(user_input.upper(), self.master_table.SitusFullAddress.astype(str), n=n_suggestions, cutoff = thresh)
       
        return suggestions

    def generate_flagged_image(image,structure_overlap, parcel_overlap, structure_flag, parcel_flag, adjacency_flag = False):
        """
        Generate a color-coded image flagging all potential violations
        image: original satellite image
        structure_overlap: boolean matrix identifying tree-structure overlap
        parcel_overlap: boolean matrix identifying tree-parcel overlap
        structure_flag: boolean denoting whether there is structure overlap in the image
        parcel_flag: boolean denoting whether there is parcel overlap in the image
        """
        image = np.array(image)
        if structure_flag:
            new_redband = structure_overlap*100 + image[:,:,0]
            image[:,:,0] = new_redband
        if parcel_flag:
            new_blueband = parcel_overlap*100 + image[:,:,2]
            image[:,:,2] = new_blueband

        return image
    